In [9]:
import os  
import glob
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = False

# image_paths = glob.glob('dogImages/*/*/*.jpg')
image_paths = glob.glob('dogImages/train/098.Leonberger/*.jpg')

for i in range(len(image_paths)):
    im = Image.open(image_paths[i])
    try:
        im.save("temp.jpg")
    except:
        print("Corrupt image: ",image_paths[i])
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        im.save(image_paths[i])
        ImageFile.LOAD_TRUNCATED_IMAGES = False
    

In [4]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [11]:
??datasets.ImageFolder

In [6]:
dataset = ImageFolderWithPaths((os.getcwd() + "/dogImages/train/").replace('\\','/')) # our custom dataset
dataloader = torch.utils.data.DataLoader(dataset)

# iterate over data
for inputs, labels, paths in dataloader:
    # use the above variables freely
    print(inputs, labels, paths)

TypeError: batch must contain tensors, numbers, dicts or lists; found <class 'PIL.Image.Image'>